1. ve 2. derste Python’ın popüler pandas, matplotlib, seaborn kütüphanelerini kullanarak veri analizi ve görselleştirme çalışmaları yapmıştık, umarım keyifli bir başlangıç olmuştur.

İlk dersleri makine öğrenmesinden ziyade Python’a giriş olarak düşünebilirsiniz. 3. dersten itibaren veri setleri üzerinde çeşitli makine öğrenmesi modelleri kuracağız ve bu modellerimizin başarı oranlarını arttırmak için parametre seçimi yapacağız.

Bu makine öğrenmesi modelleri arasında insanın düşünme tarzına en yakın olanların Karar Ağaçları (Decision Tree) ve K-En Yakın Komşu (K-Nearest Neighbour) olduğunu hatta bu modellere benzer olarak işlemcimizin (CPU) hızının, eğitim setimizin (training data) kalitesinin verdiğimiz kararları ve tahmin/öngörü yeteneğimizi etkilediğini düşünüyorum.

Diğer derslerde olduğu gibi giriş ve sonuç bölümlerinde kavramları/modelleri analoji kurarak anlatmaya çalışıyorum.

Karar Ağaçları: Kışın denize girilmez, yazın denize girilir. Kış aylarındayız dolayısıyla denize girilmez.

Zihnimizdeki karar ağaçlarının derinliği yukarıdaki örneğin aksine ekseriyetle 1'den fazla olmakla birlikte modeli basit kurmak önemli.

Ana sınıfına giden bir çocuğun zihninde böyle bir karar ağacı olabilirken insan kaynakları departmanında çalışan bir kişinin zihninde aşağıdaki gibi bir karar ağacı olmasını bekleyebiliriz.

Karar Ağaçları: Eğer bir kişi x-y yaş aralığındaysa a, b, c, d, e okullarından mezunsa en az t yıl tecrübesi varsa ve daha önceki işlerinde çalışma süresi ortalama p yıldan fazlaysa mülakata çağıralım.

Karar Ağaçları entropi (the degree of chaos in the system) değerini düşüren seçimler yaparak bilgi kazancını (information gain) maksimize etmeye çalışır.

Bizdeki karşılığı için şöyle bir oyun kurabiliriz. Örneğin 1'den 100'e kadar bir sayıyı 5 soruda bulmamız bekleniyorsa “sayı 50'den büyük mü?” sorusunun cevabı “sayı 90 mı?” sorusunun cevabından daha çok bilgi kazanmamızı sağlar.

K-En Yakın Komşu: İnsan kendine benzeyeni sever, kendine benzeyenle ortak olur, dost olur, sevgili olur.

K-En Yakın Komşu Modelini sınıf çatışmasına veya “öteki” kavramına kadar bağlama potansiyelimiz olduğu için konuyu çok uzatmadan tartışmaları ilgili sosyal bilimlere bırakıyorum ve derse başlıyorum.

In [2]:
# ASSIGNMENT 3

#Kütüphanelerimizi çağıralım
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Part 1. İkinci Buluşma Olacak Mı?"

Amacımız karar ağaçlarının nasıl çalıştığını görmek, bu sebeple kolay bir veri seti 
üzerinde çalışacağız. Tek bir karar ağacının muhteşem sonuçlar ortaya çıkardığı 
görülmese de performansı yüksek olan Gradient Boosting ve Random Forests algoritmalarına 
temel oluşturması sebebiyle anlaşılması oldukça önemlidir.

İkili (binary) sınıflandırma yapacağımız basit veri setimizde A kişinin B kişisiyle 2. buluşmaya çıkıp çıkmayacağını inceleyeceğiz. Bu kararı verirken incelenecek 
öznitelikler dış görünüş, etkili/güzel konuşma, alkol tüketimi ve ilk buluşmada ne kadar 
para harcandığı.

In [3]:
# Veri Seti Oluşturma

def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]
  
features = ['Looks', 'Alcoholic_beverage','Eloquence','Money_spent']

#Training Data

df_train = {}
df_train['Looks'] = ['handsome', 'handsome', 'handsome', 'repulsive',
                         'repulsive', 'repulsive', 'handsome'] 
df_train['Alcoholic_beverage'] = ['yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes']
df_train['Eloquence'] = ['high', 'low', 'average', 'average', 'low',
                                   'high', 'average']
df_train['Money_spent'] = ['lots', 'little', 'lots', 'little', 'lots',
                                  'lots', 'lots']
df_train['Will_go'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])
df_train = create_df(df_train, features)
df_train

,Will_go,Looks_handsome,Looks_repulsive,Alcoholic_beverage_no,Alcoholic_beverage_yes,Eloquence_average,Eloquence_high,Eloquence_low,Money_spent_little,Money_spent_lots
0,0,1,0,0,1,0,1,0,0,1
1,1,1,0,0,1,0,0,1,1,0
2,0,1,0,1,0,1,0,0,0,1
3,1,0,1,1,0,1,0,0,1,0
4,1,0,1,0,1,0,0,1,0,1
5,0,0,1,0,1,0,1,0,0,1
6,0,1,0,0,1,1,0,0,0,1


In [6]:
#Q1. Başlangıç durumunun (So) entropisi nedir?

import math
p1 = 3/7
p2 = 4/7
S0 = -p1*math.log2(p1) -p2*math.log2(p2)
print(round(S0,4))

0.9852


Q2. Verimizi "Looks_handsome" özniteliğine göre ayıralım.
- a. Soldaki grubun (S1--"Looks_handsome == 1") entropisi nedir?
- b. Sağdaki grubun (S2--"Looks_handsome == 0") entropisi nedir?
- c. Eğer böyle bir ayrım yaparsak elde edeceğimiz bilgi kazancı nedir?

In [8]:
#S1 (4 handsome 1 will_go 3 wont_go)
p1 = 3/4
p2 = 1/4
S1 = -p1*math.log2(p1) -p2*math.log2(p2)
print(round(S1,4))

#S2 (3 not_handsome 2 will_go 1 wont_go)
p1 = 2/3
p2 = 1/3
S2 = -p1*math.log2(p1) -p2*math.log2(p2)
print(round(S2,4))

IG = S0 - (4/7 * S1) - (3/7 * S2)
print(round(IG,4))

0.8113
0.9183
0.1281


In [9]:
#Eğer her durumdaki entropiyi yukardaki gibi tek tek hesaplamak istemezseniz 
#aşağıdaki fonksiyon tanımlanabilir.

p = df_train['Will_go'].value_counts()/len(df_train)
def ent(data):
    p = data.value_counts()/len(data)
    entropy = -p[0]*math.log2(p[0]) - p[1]*math.log2(p[1])
    return entropy
   
ent0 = ent(df_train['Will_go'])
ent1 = ent(df_train[df_train.Looks_handsome == 1]['Will_go'])
ent2 = ent(df_train[df_train.Looks_handsome == 0]['Will_go'])
df1 = df_train[df_train.Looks_handsome == 1]
df2 = df_train[df_train.Looks_handsome == 0]

IG = ent0 - (len(df1)/(len(df1) + len(df2)) *ent1) -  (len(df2)/(len(df1) + len(df2)) * ent2) 

print(round(ent0, 4))
print(round(ent1, 4))
print(round(ent2, 4))
print(round(IG, 4))

0.9852
0.8113
0.9183
0.1281


In [10]:
from ipywidgets import Image
from io import StringIO
import pydotplus
from sklearn.tree import export_graphviz

age_tree = DecisionTreeClassifier(max_depth = 3, random_state = 17)
age_tree.fit(df_train['Looks_handsome'].values.reshape(-1,1), df_train['Will_go'].values)
dot_data = StringIO()
export_graphviz(age_tree, feature_names = ['Looks_handsome'],
               out_file = dot_data, filled = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value = graph.create_png())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01P\x00\x00\x00\xdc\x08\x02\x00\x00\x00\xa9\x8fn\x99\x00\x00\x00\x06bKGD\x00\xff\x00\xff\x00\xff\xa0\xbd\xa7\x93\x00\x00 \x00IDATx\x9c\xed\x9dw<U\xff\x1f\xc7\xdf\xf6\xde\xa1\x8d\xd2\xd0\x12BKE\x89\xeak\xb4\xfb6\xb5\xd1\x94JTh(\xa54444TJJJ2*{\x0b\xd1T\xbf\xa8ds\xed\xcd\xf5\xfb\xe3\xf4=\xdd\xee\xe5\xba\xeet\xf9<\x1f\xfep\xde\xe7\xf3\xf9\x9c\xf7\xe7\xdc\xfb\xbe\x9fu>\xaf\xc3\xd3\xd6\xd6\x06\x08\x04\xa2w\xc0\xcbi\x07\x10\x08\x04\xfb@\x01\x8f@\xf4"P\xc0#\x10\xbd\x08~v^\xac\xa1\xa1\xe1\xc5\x8b\x17\xad\xad\xad\xec\xbc(\x02\xd1=\xe9\xdb\xb7\xaf\x9e\x9e\x1e\xbb\xaf\xda\xc6F\x1e?~\xcc\xee\xea!\x10\xdd\x15~~~vF\x1f\x06[[\xf8\x96\x96\x16\x00h\xc8Ie\xe7E\x11\x88n\xc8\xa3\xa0\x97+\xb7\xeec\xffu\xd1\x18\x1e\x81\xe8E\xa0\x80G z\x11(\xe0\x11\x88^\x04\nx\x04\xa2\x17\x81\x02\x1e\x81\xe8E\xb0u\x96\x9eF\x84U\xb4\xc8,,\x9a\xd8\xc7/\xc4\xa5\xe5w\xcfKs)\xafc\x93N\\\xf4J\x7f\xf7\t\x004\xc7\x8d\xb2\xdb\xb2\xce`\x8a.\xf5,\x94_T\xe8\xf67\xbc;\xb6\xf0\r9\xa9\xf8]#\xfd\x9f\x15\x17bQ\xc9\xec)\xbf{^\x9a\x1b\xb9\xfb\xf8\xf9\xbcU\xd6cF\x0c\xfb\x1c\x13\xf89&p\xf4p\xd5y\xab\xb6\xdc\x0fx\xc1i\xbf\x98Owl\xe1\x11\x08vRX\\\xba\xc3\xd1UWs\xdci\xa7\xdd<<<\x00p\xdai\xf7\x9b\xcc\x0f\xdb\x0f\x1c\x9f9u\xa2B\x1fY*y\xb9\xee\x87\xb5;\xb6\xf0\x08DG\xe4\xe6\x17\x9e\xf2\xbc5\xc1x)\x13\xcb\xbc\xf90\xa0\xb6\xae\xdeb\x89\x19\x16\xed\x00\xc0\xc3\xc3c\xb1\xc4\xac\xba\xb6\xee\xd6\xc3\xa7L\xbcPw\x00\x05<\x82\x0b TT]\xf7y<s\xc9\x86\xe1S\xffq:uQQ^\x8e\x89\x85\x87\xc7&\x03\x80\xf6\xf81\xa4F\xec\xf0UL\x02\x13/\xd4\x1d\xe0\xee.}QI\xd9\xe13\x9e\xc1\x11\xb1%\xa5\x04\xf9>\xb2s\r\xf4\x1cm,I\xfb`\x9d& e\xb2\xe9\xca\xb4w\x9f\xb0\xff\x17\xff3\xfb\x8e\xc7q\x00\xa8\xac\xae9z\xf6J\xe0\xcb\xa8\x82\xa2\x121Q\xe1aC\x94\'i\x8d[8o\xb6\xb6\xfah\xda\xfd\xfcUP\xb4\xd3\xf1Dd|\x8a\x88\x88\xf0,=\xdd\xd3\x8e{de\xa4\xf0\xb3\xe1qI\x17o>\x88KI\xafohT\x1b\xa6\xb2k\xf3\x9a%&F\xf8Y|f\xe8\x7f\xf1/\xa8\x14\xf2\xf1\xcb7\xfb\xe3\xe7b\x93\xd3\xf8xygL\xd6>\xed\xb4\x87\xcc\x87N+B\xfd^\xe1n\xe4$\x86\xd88\x9f|\x15\x9d((\xc8?\xc7`\x9a\xbb\xf3\x9e\x8a\xca\xaa]\xcen\xd1\x89oDD\x84gO\x9f|\xcaq\xb7\xb4\xa4\x04~\xdd\x922\xc2\xe13W\x82^G\x97\x94\x12\xfa\xc8\xc9\xcc\xd1\x9f\xea\xb4\xcb\x8a\xc6\x88\xadoh|\xfe*\xeaA@pXT|sK\x8b\xe6X\xb5\x93\x07v-11"\xcb\xde\xee\xe4\x19\x19T\xfa\xde\x9f\xff\x97\x03\x00\x03\xfb)\x92\x1a\x07\xf5\xef\x0b\x00Y\xdf\xbeS/\xd6\xe9\xd4\xc5\x07OC\n\x8aJ\xe4\xe5d\x8d\xf5\xa7\x1c\xdc\xb9\xb9\xafB\x9fN\x9d\xe1 \\\x1c\xf0E%eS\xccV\xb5\x12\x897\xdd\x8fh\xa9\x8f~\xf3\xf6\xbd\x85\xcd\x81\xb0\xa8\xf8\xd8\x00o\xeck\xdai\x022\x9ex\x9d\x9b\xbb\xd2j\x8e\x81\xdeQ\xbbm\xb8q\x83\xadS\xe0\xcb\xc8S\x8e\xbb\xd7.5\x17\x10\xe0\xff\x9e\x9b\x7f\xf0\xa4\x87\x9e\xf9\xea.\r\xde\x0e\x9e\xf48j\xb7\xad\x9f\xa2\xbc\xf3\xa9K\x97\xbd}\xf9\xf9\xf9\xaf\xb99\xe3g\xe7\xae\xb46\x9d=\xe3Cd@]}\x83\xa5\xdd\xe1\xd5\xdb\x1dd\xa4$\r\xa7M\xc2\xce6\xe4\xa4b_h\xbc\x90\x83\'=\xae\xde}DZH\xf6\x8f_\x06\x8b\xd7\x8b\x88\x08?\xbevFK}t\xc6\x87\xcf\xd6\xf6G\xc9|\xa0^\x91N\xef\x15\xee\x86\x83\xebyg[\xeb+\'\x9d\x9cN]\xf4\xf4~H\xa8\xa8\x10\x14\x10p\xd9\xb7\x1dwLP@\xe0\xd2\xf1\x03\xd8E\x8bK\tS\xcdW746\xdep?2Qk\\\xc6\x87\xac\xb56\x07\xc3\xe3\x92\x13\x9f\xdf#\xfdQ \xa3\xb5\x95\x18\x1e\x97t? \xf8YhDM]\x9d\xca\xe0\x01\xbb--\xfe\x9d?w\xf8\x10\xa5v\xd338\x90\xae\xac\xaa\x06\x00q1QR#vXQYM%\xa3\x84\x98\xe8\x90\xc1\x03\x13\x9f\xdfkk\x83\xa0WQ6N\'\x83#b\xe3\x02\xbc\xfb)\xca3\xe2\x0fK\xe1\xe2.\xfd!\xf7\xcb\xbf\n\x8a\x8e\xed\xdb>c\xb2\xb6\x84\x98\xa8\xfe\x14\x1d\x17\xbb\xed?\xf3\n\x0e\x9f\xf1\xa41\x01)?\xf3\n\x0c\x16\xaf_j6\x874\xda\x01 *!\x05\x00\xfa\xf7\x95\x17\x13\x15\x11\x14\x10\x18>D\xe9\xec!\xbb\xae\xba\xban\xd9\xfc\x91\xaa*R\x12\xe2{\xac,\x00\xe0UL"Y\x02\xb7\x83\xb6r2\xd2\x83\xfa\xf7uw\xde\x0b\x00\xae\x17\xbc\xa8\x14b\xbby\rY!G\xce^\xa9\xa8\xaa\xc6k:UGs\xe3\x8aEd\xd9\xa9W\x84\xf6{\xb5n\x999\xe6\x86\x9d\xf5:\x00\x08\x0e\x8f\xdd\xba\xf6\xdf\xdf\x96-\xeb\x00 $2\x0eO|\xf8\x8c\xe7\xcf\xbc\x82#{\xb6\xce\xd2\x9b(.*:E[\xc3\xed\xa0\xed\xf7\xdc\xbc3W\xbd\xa9\xdc.e]#\x935[C#\xe3V,\x9c\x17\xf5\xf8\xe6\xa7\xa8gN\xb6V\x1dE;\x07)y

In [14]:
# Part 2. Entropi ve Bilgi Kazancı Hesaplamaları

#9 mavi ve 11 sarı topumuz var. Mavi olanları 1 diğerlerini 0 
#olarak etiketleyelim.

balls = [1 for i in range(9)] + [0 for i in range(11)]

#Sonra bu topları soldaki gruba 8 mavi 5 sarı, sağdaki gruba 1 mavi 
#6 sarı olacak şekilde ayıralım.

balls_left  = [1 for i in range(8)] + [0 for i in range(5)]
balls_right = [1 for i in range(1)] + [0 for i in range(6)] 

#2.1. Başlangıç durumunun, soldaki, sağdaki grupların entropisi nedir?

import collections
c = collections.Counter(balls)

p1 = c[0]/len(balls)
p2 = c[1]/len(balls)
S0 = -p1*math.log2(p1) - p2*math.log2(p2)

r = collections.Counter(balls_right)
p1 = r[0]/len(balls_right)
p2 = r[1]/len(balls_right)
S1 = -p1*math.log2(p1) - p2*math.log2(p2)

l = collections.Counter(balls_left)
p1 = l[0]/len(balls_left)
p2 = l[1]/len(balls_left)
S2 = -p1*math.log2(p1) - p2*math.log2(p2)

print(round(S0, 4))
print(round(S1, 4))
print(round(S2, 4))

0.9928
0.5917
0.9612


In [15]:
#2.2. Hilesiz bir zarın entropisi nedir?

p1 = 1/6
zar = (-p1*math.log2(p1)) * 6
print(round(zar, 4))

#2.3. Başlangıç verisetini iki gruba ayırmayla elde edilen 
#bilgi kazancı nedir?

IG = S0 - (7/20 * S1) - (13/20 * S2)
print(round(IG, 4))

2.585
0.1609


In [17]:
#Part 3. "Adult" Veri Seti

#Train ve test verilerimizi github üzerinden çekiyoruz.

data_train = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/adult_train.csv',
                         sep = ';')

data_test = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/adult_test.csv', 
                        sep = ';')

#Değişkenlerimizi makine öğrenmesi algoritmalarına sokabilmek 
#için nümerik değerlere çevirmeliyiz.

data_train.at[data_train['Target'] == ' <=50K', 'Target'] = 0
data_train.at[data_train['Target'] == ' >50K', 'Target'] = 1

data_test.at[data_test['Target'] == ' <=50K.', 'Target'] = 0
data_test.at[data_test['Target'] == ' >50K.', 'Target'] = 1

#Temel Veri Analizi

data_train['Target'].value_counts()

0    24720
1     7841
Name: Target, dtype: int64

In [18]:
data_test = data_test[1:]

In [19]:
#Öznitelik Mühendisliği

print(data_train.dtypes)
print(data_test.dtypes)

Age                int64
Workclass         object
fnlwgt             int64
Education         object
Education_Num      int64
Martial_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_per_week     int64
Country           object
Target             int64
dtype: object
Age                object
Workclass          object
fnlwgt            float64
Education          object
Education_Num     float64
Martial_Status     object
Occupation         object
Relationship       object
Race               object
Sex                object
Capital_Gain      float64
Capital_Loss      float64
Hours_per_week    float64
Country            object
Target             object
dtype: object


In [20]:
#Test veri setinde yaş özniteliğinin tipinin 'object' olduğunu görüyoruz, bunu değiştirmeliyiz.
#data_test['Age'] = data_test['Age'].astype(int)

#Ayrıca train ve test veri setlerinde ikilik olmaması için bütün 'float' veri tiplerini 
#'int' veri tiplerine çeviriyoruz.

data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

#Eksik veriler için nümerik öz nitelikleri medyan değerleriyle kategorik 
#öznitelikleri mod değerleriyle dolduruyoruz.

categorical_columns_train = [c for c in data_train.columns 
                             if data_train[c].dtype.name == 'object']
numerical_columns_train = [c for c in data_train.columns 
                           if data_train[c].dtype.name != 'object']

categorical_columns_test = [c for c in data_test.columns 
                            if data_test[c].dtype.name == 'object']
numerical_columns_test = [c for c in data_test.columns 
                          if data_test[c].dtype.name != 'object']

for c in categorical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].mode())
for c in categorical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].mode())
    
for c in numerical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].median())
for c in numerical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].median())
    
#Kategorik özniteliklerden kukla (dummy) değişkenler oluşturmak için 
#pandas kütüphanesindeki get_dummies() fonksiyonunu kullanacağız.

#Değiştireceğimiz kategorik değişkenler: Workclass, Education, Martial_Status, 
  #Occupation, Relationship, Race, Sex, Country.
  
data_train = pd.concat([data_train, pd.get_dummies(data_train['Workclass'], 
                                                   prefix="Workclass"),
                      pd.get_dummies(data_train['Education'], prefix="Education"),
                      pd.get_dummies(data_train['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_train['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_train['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_train['Race'], prefix="Race"),
                      pd.get_dummies(data_train['Sex'], prefix="Sex"),
                      pd.get_dummies(data_train['Country'], prefix="Country")],
                     axis=1)

data_test = pd.concat([data_test, pd.get_dummies(data_test['Workclass'], prefix="Workclass"),
                      pd.get_dummies(data_test['Education'], prefix="Education"),
                      pd.get_dummies(data_test['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_test['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_test['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_test['Race'], prefix="Race"),
                      pd.get_dummies(data_test['Sex'], prefix="Sex"),
                      pd.get_dummies(data_test['Country'], prefix="Country")],
                     axis=1)

data_train.drop(['Workclass', 'Education', 'Martial_Status',
                 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'],
                axis=1, inplace=True)
data_test.drop(['Workclass', 'Education', 'Martial_Status', 'Occupation', 
                'Relationship', 'Race', 'Sex', 'Country'],
               axis=1, inplace=True)

In [22]:
#Veri tiplerini düzenledik, eksik verileri uygun değerlerle doldurduk, 
#kukla (dummy) değişkenlerimizi oluşturduk. Modelimizi kurmadan test ve 
#training verileri arasında bir farklılık olup olmadığını kontrol edelim.

print(set(data_train.columns) - set(data_test.columns))
print('')
print(data_train.shape, data_test.shape)


{'Country_ Holand-Netherlands'}

(32561, 106) (16281, 105)


In [25]:
#Test veri setinde Hollando yok, 0'lardan meydana gelen bir seri oluşturuyoruz.
data_test['Country_ Holand-Netherlands'] = np.zeros([data_test.shape[0], 1])

#Şimdi fark kümesinin boş olmasını bekleyebiliriz.

set(data_train.columns) - set(data_test.columns)
set()

set()

In [24]:
data_train.head()

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Portugal,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#Modelimizi kurmadan önce train ve test verimizdeki X ve y değerlerini ayırmalıyız.

X_train=data_train.drop(['Target'], axis=1)
y_train = data_train['Target']

X_test=data_test.drop(['Target'], axis=1)
y_test = data_test['Target']

#3.1 Karar Ağacı (Parametre Ayarı Olmaksızın - without parameter tuning)

#Karar ağacımızı parametre ayarı(tuning) yapmadan en fazla 3 derinlikte ve 
#random_state = 17 olacak şekilde eğitmemiz ve doğruluk (accuracy) skorunu 
#bulmamız isteniyor.

tree = DecisionTreeClassifier(max_depth = 3, random_state=17)
tree.fit(X_train, data_train['Target'].values)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [27]:
#Eğitilmiş veri setini kullanarak test verisi üzerinde tahmin yapacağız şimdi de.

tree_predictions = tree.predict(X_test)
print(round(accuracy_score(y_test, tree_predictions), 4))

0.8448


In [28]:
#Karar ağacımızı parametreleri tune etmeden en fazla 9 derinlikte ve 
#random_state = 17 olacak şekilde eğitmemiz ve doğruluk (accuracy) 
#skorunu bulmamız isteniyor.

tree = DecisionTreeClassifier(max_depth = 9, random_state=17)
tree.fit(X_train, data_train['Target'].values)
tree_predictions = tree.predict(X_test)
print(round(accuracy_score(y_test, tree_predictions), 4))

0.8472


In [29]:
#3.2 Karar Ağacı (Parametre Ayarlı - with parameter tuning)
#5-fold cross-validation tekniğini kullanarak karar ağacındaki 
#optimal değeri araştıracağız. (GridSearchCV)

tree_params = {'max_depth': range(2,15)}

locally_best_tree = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)                     

locally_best_tree.fit(X_train, data_train['Target'].values)

Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    1.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(2, 15)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=True)

In [30]:
print(locally_best_tree.best_params_) 
print(round(locally_best_tree.best_score_, 4))
print(round(accuracy_score(y_test, locally_best_tree.predict(X_test)), 4))

{'max_depth': 9}
0.8563
0.8472


1. ve 2. kısımda sentetik veri üretmeyi, entropi, bilgi kazancı hesabı yapmayı ve basit bir karar ağacını nasıl oluşturacağımızı öğrendik.
3. kısım ise standart bir veri bilimi projesinin en basit hali olarak düşünülebilir. Yaptıklarımızı şöyle yazabiliriz:

- Açıklayıcı veri analizi ve görselleştirme
- Veri tiplerini kontrol etme/düzeltme
- Eksik verileri tahmin etme/veri atama
- Kategori tipindeki verileri nümerik formata çevirme
- Test ve train veri setlerine ayırma (bu veride önceden ayrılmış.)
- Test ve train veri setlerinde farklılıkları kontrol etme
- Modelimizi eğitme ve tahmin yapma
- Modelde parametre araştırması (hyperparameter tuning) yapma
- Yukarıda yazdıklarım hemen her çalışmada gözü kapalı yapılması gereken adımlar. Probleme ve veri tipine göre bunlara ek olarak çeşitli çalışmalar yapılması mutlaka gerekecektir.

# A year spent in artificial intelligence is enough to make one believe in God.
## — Alan Perlis